In [1]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")

In [2]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in G:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)

In [3]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [4]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [5]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost: 
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", eps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [6]:

def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

In [7]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G 
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

In [8]:
G={(1,2),(1,3),(2,4),(2,5),(3,4),(3,5),(4,6),(5,6)}

In [9]:
Initialise(G)

All values of c,phi and l initialised to zero


In [10]:
c[1][2]=3; c[1][3]=5; c[2][4]=3; c[2][5]=3 
c[3][5]=5; c[3][4]=5; c[4][6]=2; c[5][6]=5

In [11]:
l[1][2]=5; l[1][3]=3; l[2][4]=3 
l[2][5]=6; l[3][4]=5 
l[3][5]=9

In [12]:
Flows(G)

((1, 2), ' has flow ', 0)
((1, 3), ' has flow ', 0)
((4, 6), ' has flow ', 0)
((5, 6), ' has flow ', 0)
((2, 5), ' has flow ', 0)
((3, 4), ' has flow ', 0)
((2, 4), ' has flow ', 0)
((3, 5), ' has flow ', 0)
('Total Cost is ', 0)


In [13]:
Iterate(G)

('Incremental Network:', set([(1, 2), (1, 3), (4, 6), (5, 6), (2, 5), (3, 4), (2, 4), (3, 5)]))
('Paths from source to sink: ', set([(1, 2, 5, 6), (1, 3, 5, 6), (1, 2, 4, 6), (1, 3, 4, 6)]))
('A best path in Gp is ', (1, 2, 4, 6), ' of minimum cost ', 8)
('The min capacity on this path is epsilon ', 2)
('The min cost is ', 8)
((1, 2), ' flow = ', 2)
((1, 3), ' flow = ', 0)
((4, 6), ' flow = ', 2)
((5, 6), ' flow = ', 0)
((2, 5), ' flow = ', 0)
((3, 4), ' flow = ', 0)
((2, 4), ' flow = ', 2)
((3, 5), ' flow = ', 0)


In [14]:

Iterate(G)

('Incremental Network:', set([(1, 2), (6, 4), (1, 3), (5, 6), (2, 1), (4, 2), (2, 5), (3, 4), (2, 4), (3, 5)]))
('Paths from source to sink: ', set([(1, 2, 5, 6), (1, 3, 4, 2, 5, 6), (1, 3, 5, 6)]))
('A best path in Gp is ', (1, 2, 5, 6), ' of minimum cost ', 11)
('The min capacity on this path is epsilon ', 5)
('The min cost is ', 11)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 0)
((4, 6), ' flow = ', 2)
((5, 6), ' flow = ', 1)
((2, 5), ' flow = ', 1)
((3, 4), ' flow = ', 0)
((2, 4), ' flow = ', 2)
((3, 5), ' flow = ', 0)


In [15]:

Iterate(G)

('Incremental Network:', set([(6, 4), (1, 3), (5, 6), (5, 2), (2, 1), (4, 2), (2, 5), (3, 4), (2, 4), (6, 5), (3, 5)]))
('Paths from source to sink: ', set([(1, 3, 4, 2, 5, 6), (1, 3, 5, 6)]))
('A best path in Gp is ', (1, 3, 4, 2, 5, 6), ' of minimum cost ', 11)
('The min capacity on this path is epsilon ', 4)
('The min cost is ', 11)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 2)
((4, 6), ' flow = ', 2)
((5, 6), ' flow = ', 3)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 2)
((2, 4), ' flow = ', 0)
((3, 5), ' flow = ', 0)


In [16]:
Iterate(G)

('Incremental Network:', set([(6, 4), (1, 3), (5, 6), (5, 2), (3, 1), (2, 1), (4, 3), (3, 4), (2, 4), (6, 5), (3, 5)]))
('Paths from source to sink: ', set([(1, 3, 5, 6)]))
('A best path in Gp is ', (1, 3, 5, 6), ' of minimum cost ', 12)
('The min capacity on this path is epsilon ', 2)
('The min cost is ', 12)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 4)
((4, 6), ' flow = ', 2)
((5, 6), ' flow = ', 5)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 2)
((2, 4), ' flow = ', 0)
((3, 5), ' flow = ', 2)


In [17]:
# Find the maximal flow through the network for minimal total travel
time for all care from A to B.

SyntaxError: invalid syntax (<ipython-input-17-b6e7bb7c840c>, line 2)

In [18]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")

In [19]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in G:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)

In [20]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [21]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [22]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [23]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost: 
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", eps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [24]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

In [25]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G 
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

In [26]:
G={(1,2),(1,3),(2,4),(2,5),(3,4),(3,7),(4,6),(5,6),(5,8),(6,7),(7,8)}

In [27]:
Initialise(G)

All values of c,phi and l initialised to zero


In [28]:
c[1][2]=5; c[1][3]=4; c[2][4]=4; c[2][5]=3 
c[3][7]=3; c[3][4]=5; c[4][6]=5; c[5][6]=2
c[5][8]=6; c[6][7]=4; c[7][8]=4

In [29]:
l[1][2]=5; l[1][3]=3; l[2][4]=3 
l[2][5]=6; l[3][4]=5; l[3][7]=9
l[4][6]=5; l[5][6]=2; l[5][8]=6
l[6][7]=4; l[7][8]=4

In [30]:
Flows(G)

((1, 2), ' has flow ', 0)
((1, 3), ' has flow ', 0)
((6, 7), ' has flow ', 0)
((4, 6), ' has flow ', 0)
((5, 6), ' has flow ', 0)
((7, 8), ' has flow ', 0)
((3, 7), ' has flow ', 0)
((2, 5), ' has flow ', 0)
((3, 4), ' has flow ', 0)
((2, 4), ' has flow ', 0)
((5, 8), ' has flow ', 0)
('Total Cost is ', 0)


In [31]:
Iterate(G)

('Incremental Network:', set([(1, 2), (1, 3), (6, 7), (4, 6), (5, 6), (3, 4), (3, 7), (2, 5), (7, 8), (2, 4), (5, 8)]))
('Paths from source to sink: ', set([(1, 3, 7, 8), (1, 3, 4, 6, 7, 8), (1, 2, 4, 6, 7, 8), (1, 2, 5, 6, 7, 8), (1, 2, 5, 8)]))
('A best path in Gp is ', (1, 3, 7, 8), ' of minimum cost ', 16)
('The min capacity on this path is epsilon ', 3)
('The min cost is ', 16)
((1, 2), ' flow = ', 0)
((1, 3), ' flow = ', 3)
((6, 7), ' flow = ', 0)
((4, 6), ' flow = ', 0)
((5, 6), ' flow = ', 0)
((7, 8), ' flow = ', 3)
((3, 7), ' flow = ', 3)
((2, 5), ' flow = ', 0)
((3, 4), ' flow = ', 0)
((2, 4), ' flow = ', 0)
((5, 8), ' flow = ', 0)


In [32]:
Iterate(G)

('Incremental Network:', set([(1, 2), (7, 3), (1, 3), (6, 7), (4, 6), (5, 6), (3, 4), (3, 1), (8, 7), (2, 5), (7, 8), (2, 4), (5, 8)]))
('Paths from source to sink: ', set([(1, 3, 4, 6, 7, 8), (1, 2, 4, 6, 7, 8), (1, 2, 5, 6, 7, 8), (1, 2, 5, 8)]))
('A best path in Gp is ', (1, 2, 5, 8), ' of minimum cost ', 17)
('The min capacity on this path is epsilon ', 3)
('The min cost is ', 17)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 3)
((6, 7), ' flow = ', 0)
((4, 6), ' flow = ', 0)
((5, 6), ' flow = ', 0)
((7, 8), ' flow = ', 3)
((3, 7), ' flow = ', 3)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 0)
((2, 4), ' flow = ', 0)
((5, 8), ' flow = ', 3)


In [33]:
Iterate(G)

('Incremental Network:', set([(1, 2), (7, 3), (1, 3), (6, 7), (4, 6), (5, 6), (3, 4), (3, 1), (7, 8), (2, 1), (8, 5), (8, 7), (5, 2), (2, 4), (5, 8)]))
('Paths from source to sink: ', set([(1, 3, 4, 6, 7, 8), (1, 2, 4, 6, 7, 8)]))
('A best path in Gp is ', (1, 3, 4, 6, 7, 8), ' of minimum cost ', 21)
('The min capacity on this path is epsilon ', 1)
('The min cost is ', 21)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 4)
((6, 7), ' flow = ', 1)
((4, 6), ' flow = ', 1)
((5, 6), ' flow = ', 0)
((7, 8), ' flow = ', 4)
((3, 7), ' flow = ', 3)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 1)
((2, 4), ' flow = ', 0)
((5, 8), ' flow = ', 3)


In [34]:
Iterate(G)

('Incremental Network:', set([(1, 2), (6, 4), (6, 7), (4, 6), (7, 3), (5, 6), (3, 4), (7, 6), (3, 1), (2, 1), (8, 5), (8, 7), (4, 3), (5, 2), (2, 4), (5, 8)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 7, ' with minimal cost ', 120)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 4)
((6, 7), ' flow = ', 1)
((4, 6), ' flow = ', 1)
((5, 6), ' flow = ', 0)
((7, 8), ' flow = ', 4)
((3, 7), ' flow = ', 3)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 1)
((2, 4), ' flow = ', 0)
((5, 8), ' flow = ', 3)


In [35]:
Iterate(G)

('Incremental Network:', set([(1, 2), (6, 4), (6, 7), (4, 6), (7, 3), (5, 6), (3, 4), (7, 6), (3, 1), (2, 1), (8, 5), (8, 7), (4, 3), (5, 2), (2, 4), (5, 8)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 7, ' with minimal cost ', 120)
((1, 2), ' flow = ', 3)
((1, 3), ' flow = ', 4)
((6, 7), ' flow = ', 1)
((4, 6), ' flow = ', 1)
((5, 6), ' flow = ', 0)
((7, 8), ' flow = ', 4)
((3, 7), ' flow = ', 3)
((2, 5), ' flow = ', 3)
((3, 4), ' flow = ', 1)
((2, 4), ' flow = ', 0)
((5, 8), ' flow = ', 3)


In [ ]:
#Part2 

In [36]:
G={(1,2),(1,4),(2,3),(3,5),(4,3),(5,6),(5,7),(6,8),(7,8),(7,2)}

In [37]:
Initialise(G)

All values of c,phi and l initialised to zero


In [38]:
c[1][2]=4; c[1][4]=6; c[2][3]=4
c[3][5]=5; c[4][3]=2; c[5][6]=4
c[5][7]=5; c[6][8]=5; c[7][8]=4
c[7][2]=4

In [39]:
l[1][2]=1; l[1][4]=3; l[2][3]=3
l[3][5]=2; l[4][3]=3; l[5][6]=3
l[5][7]=1; l[6][8]=3; l[7][8]=2
l[7][2]=2

In [40]:
Flows(G)

((1, 2), ' has flow ', 0)
((6, 8), ' has flow ', 0)
((5, 6), ' has flow ', 0)
((5, 7), ' has flow ', 0)
((1, 4), ' has flow ', 0)
((2, 3), ' has flow ', 0)
((4, 3), ' has flow ', 0)
((7, 8), ' has flow ', 0)
((7, 2), ' has flow ', 0)
((3, 5), ' has flow ', 0)
('Total Cost is ', 0)


In [41]:
Iterate(G)

('Incremental Network:', set([(1, 2), (6, 8), (5, 6), (5, 7), (1, 4), (2, 3), (4, 3), (7, 8), (7, 2), (3, 5)]))
('Paths from source to sink: ', set([(1, 4, 3, 5, 6, 8), (1, 4, 3, 5, 7, 8), (1, 2, 3, 5, 6, 8), (1, 2, 3, 5, 7, 8)]))
('A best path in Gp is ', (1, 2, 3, 5, 7, 8), ' of minimum cost ', 9)
('The min capacity on this path is epsilon ', 4)
('The min cost is ', 9)
((1, 2), ' flow = ', 4)
((6, 8), ' flow = ', 0)
((5, 6), ' flow = ', 0)
((5, 7), ' flow = ', 4)
((1, 4), ' flow = ', 0)
((2, 3), ' flow = ', 4)
((4, 3), ' flow = ', 0)
((7, 8), ' flow = ', 4)
((7, 2), ' flow = ', 0)
((3, 5), ' flow = ', 4)


In [42]:
Iterate(G)

('Incremental Network:', set([(3, 2), (6, 8), (5, 6), (5, 7), (1, 4), (7, 5), (2, 1), (8, 7), (4, 3), (5, 3), (7, 2), (3, 5)]))
('Paths from source to sink: ', set([(1, 4, 3, 5, 6, 8)]))
('A best path in Gp is ', (1, 4, 3, 5, 6, 8), ' of minimum cost ', 14)
('The min capacity on this path is epsilon ', 1)
('The min cost is ', 14)
((1, 2), ' flow = ', 4)
((6, 8), ' flow = ', 1)
((5, 6), ' flow = ', 1)
((5, 7), ' flow = ', 4)
((1, 4), ' flow = ', 1)
((2, 3), ' flow = ', 4)
((4, 3), ' flow = ', 1)
((7, 8), ' flow = ', 4)
((7, 2), ' flow = ', 0)
((3, 5), ' flow = ', 5)


In [43]:
Iterate(G)

('Incremental Network:', set([(3, 2), (6, 5), (6, 8), (5, 6), (4, 1), (4, 3), (5, 7), (1, 4), (7, 5), (2, 1), (8, 7), (8, 6), (3, 4), (7, 2), (5, 3)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 5, ' with minimal cost ', 50)
((1, 2), ' flow = ', 4)
((6, 8), ' flow = ', 1)
((5, 6), ' flow = ', 1)
((5, 7), ' flow = ', 4)
((1, 4), ' flow = ', 1)
((2, 3), ' flow = ', 4)
((4, 3), ' flow = ', 1)
((7, 8), ' flow = ', 4)
((7, 2), ' flow = ', 0)
((3, 5), ' flow = ', 5)


In [44]:
Iterate(G)

('Incremental Network:', set([(3, 2), (6, 5), (6, 8), (5, 6), (4, 1), (4, 3), (5, 7), (1, 4), (7, 5), (2, 1), (8, 7), (8, 6), (3, 4), (7, 2), (5, 3)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 5, ' with minimal cost ', 50)
((1, 2), ' flow = ', 4)
((6, 8), ' flow = ', 1)
((5, 6), ' flow = ', 1)
((5, 7), ' flow = ', 4)
((1, 4), ' flow = ', 1)
((2, 3), ' flow = ', 4)
((4, 3), ' flow = ', 1)
((7, 8), ' flow = ', 4)
((7, 2), ' flow = ', 0)
((3, 5), ' flow = ', 5)


In [ ]:
#Question 3

In [89]:
def Initialise20 (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[20 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of phi and l initialised to zero, c to 20")

In [90]:

def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in G:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)

In [91]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [92]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [93]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost: 
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", eps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [94]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

In [95]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G 
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

In [96]:
G={(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(1,11),(1,12),(1,13),(2,14),(3,15),
(4,16),(5,17),(6,18),(7,19),(8,20),(9,21),(10,22),(11,23),(12,24),(13,25),(14,3),(15,4),(16,5),
(17,6),(18,7),(19,8),(20,9),(21,10),(22,11),(23,12),(24,13),(14,26),(15,26),(16,26),
(17,26),(18,26),(19,26),(20,26),(21,26),(22,26),(23,26),(24,26),(25,26)}

In [97]:
Initialise(G)

All values of c,phi and l initialised to zero


In [98]:
c[1][2]=20; c[1][3]=20; c[1][4]=20; c[1][5]=20; c[1][6]=20; c[1][7]=20; c[1][8]=20; c[1][9]=20
c[1][10]=20; c[1][11]=20; c[1][12]=20; c[1][13]=20; c[2][14]=20; c[3][15]=20; c[4][16]=20; c[5][17]=20
c[6][18]=20; c[7][19]=20; c[8][20]=20; c[9][21]=20; c[10][22]=20; c[11][23]=20; c[12][24]=20; c[13][25]=20
c[14][3]=20; c[15][4]=20; c[16][5]=20; c[17][6]=20; c[18][7]=20; c[19][8]; c[20][9]=20; c[21][10]=20
c[22][11]=20; c[23][12]=20; c[24][13]=20; c[14][26]=9; c[15][26]=6; c[16][26]=6; c[17][26]=7
c[18][26]=11; c[19][26]=14; c[20][26]=16; c[21][26]=18; c[22][26]=15; c[23][26]=10; c[24][26]=7; c[25][26]=6

In [99]:
l[1][2]=18; l[1][3]=17; l[1][4]=17; l[1][5]=15; l[1][6]=12; l[1][7]=8; l[1][8]= 7; l[1][9]=6
l[1][10]=9; l[1][11]=12; l[1][12]= 14; l[1][13]=17; l[2][14]=1; l[3][15]=1; l[4][16]=2; l[16][5]=2; l[5][17]=2
l[6][18]=3; l[7][19]=5; l[8][20]=6; l[9][21]=6; l[10][22]=5; l[11][23]=3; l[12][24]=2; l[13][25]=1
l[14][3]=1; l[15][4]=1; l[16][5]=2; l[17][6]=2; l[18][7]=3; l[19][8]=5; l[20][9]=6; l[21][10]=6
l[22][11]=5; l[23][12]=3; l[24][13]=2; l[14][26]=0; l[15][26]=0; l[16][26]=0; l[17][26]=0
l[18][26]=0; l[19][26]=0; l[20][26]=0; l[21][26]=0; l[22][26]=0; l[23][26]=0; l[24][26]=0; l[25][26]=0

In [100]:
Flows(G)

((19, 8), ' has flow ', 0)
((3, 15), ' has flow ', 0)
((1, 3), ' has flow ', 0)
((19, 26), ' has flow ', 0)
((22, 26), ' has flow ', 0)
((24, 26), ' has flow ', 0)
((18, 26), ' has flow ', 0)
((11, 23), ' has flow ', 0)
((14, 26), ' has flow ', 0)
((1, 6), ' has flow ', 0)
((23, 26), ' has flow ', 0)
((23, 12), ' has flow ', 0)
((17, 26), ' has flow ', 0)
((1, 11), ' has flow ', 0)
((13, 25), ' has flow ', 0)
((15, 26), ' has flow ', 0)
((1, 2), ' has flow ', 0)
((20, 26), ' has flow ', 0)
((17, 6), ' has flow ', 0)
((1, 5), ' has flow ', 0)
((9, 21), ' has flow ', 0)
((8, 20), ' has flow ', 0)
((25, 26), ' has flow ', 0)
((7, 19), ' has flow ', 0)
((20, 9), ' has flow ', 0)
((5, 17), ' has flow ', 0)
((1, 10), ' has flow ', 0)
((10, 22), ' has flow ', 0)
((6, 18), ' has flow ', 0)
((21, 26), ' has flow ', 0)
((1, 13), ' has flow ', 0)
((1, 4), ' has flow ', 0)
((4, 16), ' has flow ', 0)
((15, 4), ' has flow ', 0)
((1, 9), ' has flow ', 0)
((22, 11), ' has flow ', 0)
((2, 14), ' has fl

In [101]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (19, 26), (22, 26), (24, 26), (18, 26), (11, 23), (14, 26), (1, 6), (23, 26), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (20, 26), (17, 6), (1, 5), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (10, 22), (6, 18), (21, 26), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (1, 9), (22, 11), (2, 14), (16, 5), (1, 12), (18, 7), (1, 8), (21, 10), (9, 21), (14, 3), (24, 13), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 11, 23, 12, 24, 13, 25, 26), (1, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 26), (1, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 4, 16, 26), (1, 5, 17, 6, 18, 7, 19, 26), (1, 2, 14, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 8, 20, 26), (1, 3, 15, 4, 16, 26), (1, 7, 19, 26), (1, 10, 22, 11, 23, 12, 24, 13

In [102]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (19, 26), (22, 26), (24, 26), (18, 26), (11, 23), (14, 26), (1, 6), (23, 26), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (20, 26), (17, 6), (1, 5), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (1, 9), (22, 11), (2, 14), (16, 5), (1, 12), (21, 9), (18, 7), (1, 8), (21, 10), (9, 21), (14, 3), (24, 13), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 11, 23, 12, 24, 13, 25, 26), (1, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 26), (1, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 12, 24, 26), (1, 5, 17, 6, 18, 7, 19, 26), (1, 2, 14, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 8, 20, 26), (1, 3, 15, 4, 16, 26), (1, 7, 19, 26), (1, 

In [103]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (19, 26), (22, 26), (24, 26), (18, 26), (11, 23), (14, 26), (1, 6), (23, 26), (20, 8), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (1, 9), (22, 11), (2, 14), (16, 5), (1, 12), (21, 9), (18, 7), (1, 8), (21, 10), (9, 21), (14, 3), (24, 13), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 11, 23, 12, 24, 13, 25, 26), (1, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 3, 15, 4, 16, 5, 17, 6, 18, 7, 19, 26), (1, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 4, 16, 26), (1, 5, 17, 6, 18, 7, 19, 26), (1, 6, 18, 7, 19, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 3, 15, 4, 16, 26), (1, 7, 19, 26), (1, 1

In [104]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (22, 26), (24, 26), (18, 26), (11, 23), (14, 26), (1, 6), (23, 26), (20, 8), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (7, 1), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (1, 9), (22, 11), (2, 14), (16, 5), (1, 12), (21, 9), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (14, 3), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 11, 23, 12, 24, 26), (1, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 6, 18, 26), (1, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 4, 16, 26), (1, 2, 14, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 10, 22, 11, 23, 12, 24, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 3, 15, 26), (1,

In [105]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (24, 26), (18, 26), (11, 23), (14, 26), (1, 6), (23, 26), (20, 8), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (22, 10), (26, 21), (6, 18), (7, 1), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (1, 9), (10, 1), (22, 11), (2, 14), (10, 22), (16, 5), (1, 12), (21, 9), (26, 22), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (14, 3), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 5, 17, 6, 18, 26), (1, 3, 15, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 12, 24, 13, 25, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 4, 16, 5, 17, 6, 18, 26), (1, 6, 18, 26), (1, 4, 16, 26), (1, 3, 15, 4, 16, 26), (1, 3, 15, 4, 16, 5, 17, 

In [106]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (24, 26), (11, 23), (14, 26), (1, 6), (23, 26), (20, 8), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (7, 1), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (18, 6), (1, 9), (10, 1), (22, 11), (2, 14), (26, 18), (22, 10), (16, 5), (1, 12), (21, 9), (6, 1), (26, 22), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (14, 3), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 3, 15, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 12, 24, 13, 25, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 11, 23, 12, 24, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 26), (1, 12, 24, 26), (1, 3, 15, 4, 16, 26), (1, 2, 14, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26)

In [107]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (9, 1), (26, 23), (24, 26), (11, 23), (14, 26), (1, 6), (20, 8), (23, 12), (17, 26), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (11, 1), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (7, 1), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (18, 6), (1, 9), (10, 1), (22, 11), (2, 14), (26, 18), (23, 11), (22, 10), (16, 5), (1, 12), (21, 9), (6, 1), (26, 22), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (14, 3), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 3, 15, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 26), (1, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 11, 23, 12, 24, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 12, 24, 26), (1, 4, 16, 26), (1, 3, 15, 4, 16, 26), (1, 2, 14, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 5, 17, 26), (1, 9

In [108]:
Iterate(G)

('Incremental Network:', set([(26, 24), (1, 3), (12, 1), (9, 1), (3, 15), (26, 23), (11, 23), (14, 26), (1, 6), (20, 8), (23, 12), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (11, 1), (20, 9), (5, 17), (1, 10), (10, 22), (26, 21), (6, 18), (7, 1), (1, 13), (24, 12), (14, 3), (1, 4), (4, 16), (15, 4), (18, 6), (1, 9), (10, 1), (22, 11), (2, 14), (26, 18), (23, 11), (22, 10), (16, 5), (1, 12), (21, 9), (6, 1), (26, 22), (1, 7), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (17, 26), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 3, 15, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 5, 17, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 4, 16, 5, 17, 26), (1, 2, 14, 26), (1, 3, 15, 4, 16, 5, 17, 26), (1, 2, 14, 3, 15, 26), (1, 3, 15, 4, 16, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 5, 17, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 1

In [109]:
Iterate(G)

('Incremental Network:', set([(26, 24), (1, 3), (12, 1), (9, 1), (3, 15), (26, 23), (11, 23), (14, 26), (1, 6), (20, 8), (23, 12), (5, 1), (1, 11), (15, 26), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (11, 1), (20, 9), (26, 17), (5, 17), (1, 10), (19, 7), (10, 22), (17, 5), (26, 21), (6, 18), (7, 1), (1, 13), (1, 7), (1, 4), (4, 16), (15, 4), (18, 6), (1, 9), (10, 1), (22, 11), (2, 14), (26, 18), (23, 11), (22, 10), (16, 5), (1, 12), (21, 9), (6, 1), (26, 22), (18, 7), (1, 8), (21, 10), (9, 21), (24, 12), (14, 3), (24, 13), (26, 19), (12, 24), (16, 26)]))
('Paths from source to sink: ', set([(1, 3, 15, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 26), (1, 3, 15, 4, 16, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 13, 25, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 4, 16, 26)]))
('A best path in G

In [110]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (12, 1), (9, 1), (26, 24), (26, 23), (11, 23), (16, 26), (14, 26), (1, 6), (20, 8), (23, 12), (5, 1), (1, 11), (1, 2), (17, 6), (26, 20), (1, 5), (8, 1), (13, 25), (8, 20), (25, 26), (7, 19), (11, 1), (20, 9), (26, 17), (15, 3), (1, 10), (1, 7), (10, 22), (26, 15), (26, 21), (6, 18), (7, 1), (1, 13), (5, 17), (24, 12), (17, 5), (4, 16), (15, 4), (18, 6), (1, 4), (10, 1), (22, 11), (2, 14), (26, 18), (23, 11), (22, 10), (16, 5), (1, 12), (21, 9), (6, 1), (3, 1), (26, 22), (18, 7), (1, 8), (21, 10), (9, 21), (19, 7), (14, 3), (24, 13), (26, 19), (12, 24), (1, 9)]))
('Paths from source to sink: ', set([(1, 2, 14, 26), (1, 12, 24, 13, 25, 26), (1, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 8, 20, 9, 21, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 11, 23, 12, 24, 13, 25, 26), (1, 3, 15, 4, 16, 26), (1, 2, 14, 3, 15, 4, 16, 26), (1, 4, 16, 26), (1, 10, 22, 11, 23, 12, 24, 13, 25, 26), (1, 13, 25, 26)]))
('A best path in Gp is ', (1, 13, 25, 26

In [111]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (12, 1), (8, 20), (21, 9), (14, 26), (1, 6), (1, 11), (26, 25), (26, 20), (13, 25), (7, 19), (20, 9), (26, 17), (15, 4), (17, 5), (6, 18), (15, 3), (1, 4), (4, 16), (1, 9), (22, 11), (2, 14), (26, 18), (10, 22), (16, 5), (1, 12), (3, 1), (18, 7), (25, 13), (21, 10), (1, 7), (26, 24), (9, 1), (26, 23), (11, 23), (24, 13), (20, 8), (23, 12), (5, 1), (1, 2), (17, 6), (8, 1), (23, 11), (12, 24), (1, 5), (11, 1), (1, 13), (1, 10), (22, 10), (26, 15), (26, 21), (7, 1), (5, 17), (24, 12), (18, 6), (16, 26), (10, 1), (13, 1), (6, 1), (26, 22), (1, 8), (9, 21), (19, 7), (14, 3), (26, 19)]))
('Paths from source to sink: ', set([(1, 2, 14, 3, 15, 4, 16, 26), (1, 2, 14, 26), (1, 3, 15, 4, 16, 26), (1, 4, 16, 26)]))
('A best path in Gp is ', (1, 2, 14, 26), ' of minimum cost ', 19)
('The min capacity on this path is epsilon ', 6)
('The min cost is ', 19)
((19, 8), ' flow = ', 0)
((3, 15), ' flow = ', 6)
((1, 3), ' flow = ', 6)
((19, 26), ' flow = ', 14

In [112]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (12, 1), (21, 9), (26, 14), (1, 6), (1, 11), (26, 25), (14, 2), (26, 20), (13, 25), (7, 19), (20, 9), (26, 17), (15, 4), (17, 5), (6, 18), (15, 3), (1, 4), (4, 16), (1, 9), (22, 11), (2, 14), (26, 18), (10, 22), (16, 5), (1, 12), (3, 1), (18, 7), (25, 13), (21, 10), (1, 7), (12, 24), (26, 24), (9, 1), (26, 23), (2, 1), (11, 23), (24, 13), (20, 8), (23, 12), (5, 1), (1, 2), (17, 6), (8, 1), (19, 7), (8, 20), (1, 5), (11, 1), (1, 13), (1, 10), (22, 10), (26, 15), (26, 21), (7, 1), (5, 17), (18, 6), (16, 26), (10, 1), (13, 1), (23, 11), (6, 1), (26, 22), (1, 8), (9, 21), (24, 12), (14, 3), (26, 19)]))
('Paths from source to sink: ', set([(1, 2, 14, 3, 15, 4, 16, 26), (1, 4, 16, 26), (1, 3, 15, 4, 16, 26)]))
('A best path in Gp is ', (1, 4, 16, 26), ' of minimum cost ', 19)
('The min capacity on this path is epsilon ', 6)
('The min cost is ', 19)
((19, 8), ' flow = ', 0)
((3, 15), ' flow = ', 6)
((1, 3), ' flow = ', 6)
((19, 26), ' flow = ', 1

In [113]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (12, 1), (21, 9), (26, 14), (1, 6), (1, 11), (26, 25), (14, 2), (26, 20), (13, 25), (26, 16), (7, 19), (20, 9), (26, 17), (15, 4), (17, 5), (6, 18), (15, 3), (1, 4), (4, 16), (1, 9), (22, 11), (2, 14), (26, 18), (10, 22), (16, 5), (1, 12), (3, 1), (18, 7), (25, 13), (21, 10), (1, 7), (12, 24), (26, 24), (9, 1), (26, 23), (2, 1), (11, 23), (24, 13), (20, 8), (23, 12), (5, 1), (1, 2), (17, 6), (8, 1), (19, 7), (16, 4), (8, 20), (1, 5), (11, 1), (1, 13), (1, 10), (22, 10), (4, 1), (26, 15), (26, 21), (7, 1), (5, 17), (18, 6), (10, 1), (13, 1), (23, 11), (6, 1), (26, 22), (1, 8), (9, 21), (24, 12), (14, 3), (26, 19)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 125, ' with minimal cost ', 1863)
((19, 8), ' flow = ', 0)
((3, 15), ' flow = ', 6)
((1, 3), ' flow = ', 6)
((19, 26), ' flow = ', 14)
((22, 26), ' flow = ', 15)
((24, 26), ' flow = ', 7)
((18, 26), ' flow = ', 11)
((11, 23), ' flow = ', 10)
((14, 26), ' flow = ', 

In [114]:
Iterate(G)

('Incremental Network:', set([(3, 15), (1, 3), (12, 1), (21, 9), (26, 14), (1, 6), (1, 11), (26, 25), (14, 2), (26, 20), (13, 25), (26, 16), (7, 19), (20, 9), (26, 17), (15, 4), (17, 5), (6, 18), (15, 3), (1, 4), (4, 16), (1, 9), (22, 11), (2, 14), (26, 18), (10, 22), (16, 5), (1, 12), (3, 1), (18, 7), (25, 13), (21, 10), (1, 7), (12, 24), (26, 24), (9, 1), (26, 23), (2, 1), (11, 23), (24, 13), (20, 8), (23, 12), (5, 1), (1, 2), (17, 6), (8, 1), (19, 7), (16, 4), (8, 20), (1, 5), (11, 1), (1, 13), (1, 10), (22, 10), (4, 1), (26, 15), (26, 21), (7, 1), (5, 17), (18, 6), (10, 1), (13, 1), (23, 11), (6, 1), (26, 22), (1, 8), (9, 21), (24, 12), (14, 3), (26, 19)]))
('Paths from source to sink: ', set([]))
('Maximal flow found:', 125, ' with minimal cost ', 1863)
((19, 8), ' flow = ', 0)
((3, 15), ' flow = ', 6)
((1, 3), ' flow = ', 6)
((19, 26), ' flow = ', 14)
((22, 26), ' flow = ', 15)
((24, 26), ' flow = ', 7)
((18, 26), ' flow = ', 11)
((11, 23), ' flow = ', 10)
((14, 26), ' flow = ', 

In [ ]:
#c alculation for question2 

In [ ]:
#Suppose that the firm has 500kg of fruit in storage at the beginning of the year.
What is the best purchasing schedule that ensures that 500kg are again in storage
at the very end of the year

In [ ]:
def Initialise20 (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[20 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of phi and l initialised to zero, c to 20")